In [4]:
from urllib import request
from bs4 import BeautifulSoup as soup
import json
import sys

In [5]:
header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'User-Agent' : "Magic Browser"}
school_link = "https://02.ppdbjatim.net/pengumuman/pengumuman_penerimaan/smk/sekolah/2413"

In [14]:
def fetch_school(link):
    client = request.urlopen(request.Request(link, headers=header))

    data = soup(client.read(), "html.parser")

    client.close()

    students = []

    for s in data.findAll("tr")[1:]:
        detail = s.findAll("td")

        students.append({
            "urutan": int(detail[0].text),
            "no-ujian": detail[1].text,
            "nama": detail[2].text,
            "sekolah-asal": detail[3].text,
            "status": detail[4].text.strip(),
            "nilai": float(detail[5].text),
            "detail-nilai": detail[6].text,
            "waktu-daftar": detail[7].text,
            "link": detail[8].a["href"],
        })

    return students

In [54]:
def scrape_user(link, name=None):
    print("scaping user:", name or link.split("/")[-1], end ="... ")
    sys.stdout.flush()

    client = request.urlopen(request.Request(link, headers=header))
    user_soup = soup(client.read(), "html.parser")
    client.close()
    
    sma = link.split("/")[-2] == "sma"
    container = user_soup.select("body > div.container-fluid > div > div > div.row.font > div.col-md-12.col-md-border > div > div.panel-body")[0]
    kelamin = container.div.findAll("div")[8].span.text
    span = container.findAll("div", {"class": "col-md-7"})[0].findAll("span")
    
    if not sma:
        pilihan = [ 
            {
                "Nama Sekolah": span[0].text,
                "Jurusan": span[1].text,
                "Status": span[2].text
            },{
                "Nama Sekolah": span[3].text,
                "Jurusan": span[4].text,
                "Status": span[5].text
            }
        ]
    else:
        pilihan = [ 
            {
                "Nama Sekolah": span[0].text,
                "Status": span[1].text
            },{
                "Nama Sekolah": span[2].text,
                "Status": span[3].text
            }
        ]
    
    print("done!")

    return [kelamin, pilihan, span[4 if sma else 6].text]

In [51]:
def scrape(link, detail, num=None):
    data = fetch_school(link)

    print("Get {0} data".format(num))

    if not detail:
        return data[:num]

    new_data = []

    for d in data[:num]:
        detail = scrape_user(d["link"], d["nama"])

        d["kelamin"] = detail[0]
        d["pilihan"] = detail[1]
        d["diterima-sementara"] = detail[2]

        new_data.append(d)
        
    return new_data

In [55]:
data = scrape("https://09.ppdbjatim.net/pengumuman/pengumuman_penerimaan/sma/sekolah/64", True, 10)

Get 10 data
scaping user: AGAPHE SULUH BRAHMANTIO... done!
scaping user: DWI HIKMAH AMALIA... done!
scaping user: DAYANG PUSPA SANDYA FAIZA... done!
scaping user: PRETTY MELATI PARDEDE... done!
scaping user: YASSAR ANNABIL... done!
scaping user: ATIQAH MUFIDAH NUR NIANSAH... done!
scaping user: YONATHAN ADITYA GUNAWAN... done!
scaping user: MUHAMMAD NADHIF NARARYASENA KUSUMA... done!
scaping user: DHEA AMELIA PUTRI... done!
scaping user: IVONE ADELIA ROSIANI SETIAWAN... done!


In [27]:
data

[{'detail-nilai': '100.00 | 87.50 | 88.00 | 94.00',
  'diterima-sementara': 'SMA NEGERI 1 BATU ',
  'kelamin': 'LAKI-LAKI',
  'link': 'https://09.ppdbjatim.net/pengumuman/detail_pengumuman/sma/21805090010987',
  'nama': 'AGAPHE SULUH BRAHMANTIO',
  'nilai': 369.5,
  'no-ujian': '21805090010987',
  'pilihan': [{'Nama Sekolah': 'SMA NEGERI 1 BATU',
    'Status': 'Dalam Zona | Dalam Kota'},
   {'Nama Sekolah': 'SMA NEGERI 2 BATU', 'Status': 'Dalam Zona | Dalam Kota'}],
  'sekolah-asal': 'SMP NEGERI 1 BATU',
  'status': 'DK',
  'urutan': 1,
  'waktu-daftar': '2018-06-25 09:34:43'},
 {'detail-nilai': '95.00 | 87.50 | 86.00 | 96.00',
  'diterima-sementara': 'SMA NEGERI 1 BATU ',
  'kelamin': 'PEREMPUAN',
  'link': 'https://09.ppdbjatim.net/pengumuman/detail_pengumuman/sma/21805090010454',
  'nama': 'DWI HIKMAH AMALIA',
  'nilai': 364.5,
  'no-ujian': '21805090010454',
  'pilihan': [{'Nama Sekolah': 'SMA NEGERI 1 BATU',
    'Status': 'Dalam Zona | Dalam Kota'},
   {'Nama Sekolah': 'SMA NEGERI

[{'detail-nilai': '92.50 | 75.00 | 88.00 | 92.00',
  'diterima-sementara': 'SMK NEGERI 8 MALANG  REKAYASA PERANGKAT LUNAK',
  'kelamin': 'LAKI-LAKI',
  'link': 'https://02.ppdbjatim.net/pengumuman/detail_pengumuman/smk/21805020642792',
  'nama': 'ACHMAD ARIES NAZALI',
  'nilai': 347.5,
  'no-ujian': '21805020642792',
  'pilihan': [{'Jurusan': 'REKAYASA PERANGKAT LUNAK',
    'Nama Sekolah': 'SMK NEGERI 8 MALANG',
    'Status': 'Luar Zona | Dalam Kota'},
   {'Jurusan': 'TEKNIK MEKATRONIKA',
    'Nama Sekolah': 'SMK NEGERI 8 MALANG',
    'Status': 'Luar Zona | Dalam Kota'}],
  'sekolah-asal': 'SMP NEGERI 16 MALANG',
  'status': 'DK',
  'urutan': 1,
  'waktu-daftar': '2018-06-25 18:34:24'},
 {'detail-nilai': '87.50 | 80.00 | 86.00 | 94.00',
  'diterima-sementara': 'SMK NEGERI 8 MALANG  REKAYASA PERANGKAT LUNAK',
  'kelamin': 'LAKI-LAKI',
  'link': 'https://02.ppdbjatim.net/pengumuman/detail_pengumuman/smk/21805275692187',
  'nama': 'MOHAMMAD ZAIDAN AZMI',
  'nilai': 347.5,
  'no-ujian': '2

In [30]:
'https://09.ppdbjatim.net/pengumuman/detail_pengumuman/sma/21805090010987'.split("/")[-2]

'sma'

In [45]:
print("a", '', "b")

a  b


In [57]:
firenumerate([3,1,3])

In [58]:
"a" if False else "b" if False else "c" if True else "d"

'c'